In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from Levenshtein import distance as levenshtein_distance

# Dataset de URLs confiables
trusted_urls = ['youtube.com', 'google.com', 'facebook.com', 'amazon.com', 'wikipedia.org']
trusted_df = pd.DataFrame(trusted_urls, columns=['url'])

# Función para calcular el mínimo de distancia Levenshtein con URLs confiables
def min_levenshtein_distance(url, trusted_urls):
    return min([levenshtein_distance(url, trusted_url) for trusted_url in trusted_urls])

# Generar un dataset de entrenamiento con URLs maliciosas y confiables
data = {
    'url': ['youtuve.com', 'g00gle.com', 'amaz0n.com', 'example.com', 'wikipediia.org'],
    'label': [1, 1, 1, 0, 1]  # 1 = phishing, 0 = seguro
}
df = pd.DataFrame(data)

# Calcular características adicionales
df['levenshtein_min'] = df['url'].apply(lambda x: min_levenshtein_distance(x, trusted_urls))
df['special_char_count'] = df['url'].apply(lambda x: sum(not c.isalnum() for c in x))

# Separar en características (X) y etiquetas (y)
X = df[['levenshtein_min', 'special_char_count']]
y = df['label']

# Entrenar un modelo de clasificación
model = RandomForestClassifier(random_state=0)
model.fit(X, y)

# Predicción y evaluación
y_pred = model.predict(X)
accuracy = accuracy_score(y, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Prueba de una URL nueva
new_url = 'youtobe.com'
new_url_features = {
    'levenshtein_min': min_levenshtein_distance(new_url, trusted_urls),
    'special_char_count': sum(not c.isalnum() for c in new_url)
}

new_url_features_df = pd.DataFrame([new_url_features])
prediction = model.predict(new_url_features_df)

print(f"La URL '{new_url}' es {'phishing' if prediction[0] == 1 else 'segura'}")
